<a href="https://colab.research.google.com/github/Homaoa/From-scraping-to-data-analysis-Glassdoor-website/blob/main/Cleaning_the_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [41]:
url='/content/glassdoor_scrape.csv'
df=pd.read_csv(url)
df.head()

,Unnamed: 0,Title,City,Salary,Rating,Company
0,0,DATA SCIENTIST-HEALTHCARE,"Dayton, OH",$56K - $96K,3.6,Premier Health
1,1,Data Scientist,"Washington, DC",$78K - $150K,3.9,Deloitte
2,2,Data Scientist,"Lorton, VA",$120K,2.6,QinetiQ Inc.
3,3,Data Scientist with TS Clearance,"Arlington, VA",$86K - $154K,3.9,Deloitte
4,4,Data Scientist Principal,"Hagerstown, MD",$73K - $137K,4.0,JLG Equipment Services Inc


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 870 entries, 0 to 869
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  870 non-null    int64  
 1   Title       870 non-null    object 
 2   City        870 non-null    object 
 3   Salary      544 non-null    object 
 4   Rating      735 non-null    float64
 5   Company     870 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 40.9+ KB


In [ ]:
#Salary parsing

#there is missing data in salary column that should be removed.
#also, some salaries are per hour. A new variable should be defined to convey that.
#min, max and average of the salary should be calculated.

In [42]:
df.dropna(subset=['Salary'],axis=0,inplace=True)

In [43]:
df['Hourly']=df['Salary'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)

In [45]:

df['Salary without $ and K']=df['Salary'].apply(lambda x: x.replace('$',' ').replace('K',' '))
df['Salary without $ and K']=df['Salary without $ and K'].apply(lambda x: x.lower().replace('per hour',' '))

In [47]:
df['Min_Salary']=df['Salary without $ and K'].apply(lambda x: int(x.split('-')[0]))
df['Max_Salary']=df['Salary without $ and K'].apply(lambda x: int(x.split('-')[1]) if '-' in x else int(x.split('-')[0]))
df['Avg_Salary']=(df.Min_Salary+df.Max_Salary)/2

In [48]:
#hourly wage to annual. to obtain annual salary I multiple hour salary into 2000. but since other annual salaries had 1000 as K that was removed,
#it is enough to multiple hour salari into 2.
df['Min_Salary']=df.apply(lambda x: x['Min_Salary'] if x['Hourly']==0 else x['Min_Salary']*2,axis=1)
df['Max_Salary']=df.apply(lambda x: x['Max_Salary'] if x['Hourly']==0 else x['Max_Salary']*2,axis=1)
df['Avg_Salary']=df.apply(lambda x: x['Avg_Salary'] if x['Hourly']==0 else x['Avg_Salary']*2,axis=1)

In [ ]:
#City parsing

#variable state should be added, because it makes more sense to work with states instead of the cities

In [49]:
df['State']=df['City'].apply(lambda x: x.split(',')[-1])

In [50]:
df=df.drop(['Unnamed: 0'],axis=1)

In [51]:
df.reset_index(drop=True,inplace=True)

In [ ]:
#simplifying titles

In [52]:
def title_simplifier(title):
  if 'data scientist' in title.lower():
    return 'data scientist'
  elif 'data engineer' in title.lower():
    return 'data engineer'
  elif 'analyst' in title.lower():
    return 'analyst'
  elif 'machine learning' in title.lower():
    return 'ml'
  elif 'manager' in title.lower():
    return 'manager'
  elif 'director' in title.lower():
    return 'director'
  else:
    return 'na'

def seniority(title):
  if 'sr' in title.lower() or 'senior' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
    return 'sr'
  elif 'jr' in title.lower() or 'junior' in title.lower():
    return 'jr'
  else:
    return 'na'


In [53]:
df['Title Simp']=df['Title'].apply(title_simplifier)

In [54]:
df['Seniority']=df['Title'].apply(seniority)

In [55]:
#I saw that there is a job in Bristol, UK. I remove that

df['State']=df['State'].apply(lambda x: x.strip() if x.strip() != 'Bristol' else 'na')

In [57]:
df.to_csv('glassdoor_data_cleaned.csv')